In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier, StackingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import pickle

In [ ]:
df = pd.read_csv('./datasets/csgo_processed.csv', sep=';')

orig_X = df.drop(['bomb_planted'], axis=1).values
orig_y = df['bomb_planted'].values

oversampler = SMOTE()
X, y = oversampler.fit_resample(orig_X, orig_y)

ss = StandardScaler()
X = ss.fit_transform(X)
categorical_y = tf.keras.utils.to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
cat_X_train, cat_X_test, cat_y_train, cat_y_test = train_test_split(X, categorical_y, test_size=0.2, random_state=42, stratify=categorical_y)

In [ ]:
gnb = GaussianNB()
gbc = GradientBoostingClassifier(learning_rate=0.2, n_estimators=100, max_depth=5)
bc = BaggingClassifier(n_estimators=100, max_features=0.5, max_samples=0.75)
sc = StackingClassifier(cv=5, final_estimator=LogisticRegression(), estimators=[('lr', LogisticRegression()), ('rf', RandomForestClassifier()), ('svc', SVC(probability=True))])
xgbc = XGBClassifier()
nn = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
gnb.fit(X_train, y_train)
print("GNB score:", gnb.score(X_test, y_test))
gbc.fit(X_train, y_train)
print("GBC score:", gbc.score(X_test, y_test))
bc.fit(X_train, y_train)
print("BC score:", bc.score(X_test, y_test))
sc.fit(X_train, y_train)
print("SC score:", sc.score(X_test, y_test))
xgbc.fit(X_train, y_train)
print("XGBC score:", xgbc.score(X_test, y_test))

In [ ]:
nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
nn.fit(cat_X_train, cat_y_train, epochs=100, batch_size=32, validation_data=(cat_X_test, cat_y_test))
loss, accuracy = nn.evaluate(cat_X_test, cat_y_test, verbose=0)
print("NN test accuracy:", accuracy)

In [ ]:
with open('./models/gnb.pkl', 'wb') as f:
    pickle.dump(gnb, f)
with open('./models/gbc.pkl', 'wb') as f:
    pickle.dump(gbc, f)
with open('./models/bc.pkl', 'wb') as f:
    pickle.dump(bc, f)
with open('./models/sc.pkl', 'wb') as f:
    pickle.dump(sc, f)
with open('./models/xgbc.pkl', 'wb') as f:
    pickle.dump(xgbc, f)
nn.save('./models/nn_model.keras')

In [ ]:
with open('./models/gnb.pkl', 'rb') as f:
    gnb = pickle.load(f)
with open('./models/gbc.pkl', 'rb') as f:
    gbc = pickle.load(f)
with open('./models/bc.pkl', 'rb') as f:
    bc = pickle.load(f)
with open('./models/sc.pkl', 'rb') as f:
    sc = pickle.load(f)
with open('./models/xgbc.pkl', 'rb') as f:
    xgbc = pickle.load(f)
nn = tf.keras.models.load_model('./models/nn_model.keras')